In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("/home/vmeshchaninov/DiffusionTextGeneration-cond-ca/")

In [3]:
from datasets import load_from_disk, Dataset

In [4]:
dt = load_from_disk("/home/vmeshchaninov/nlp_models/data/common_gen/valid")

In [5]:
dt

Dataset({
    features: ['concept_set_idx', 'concepts', 'target'],
    num_rows: 4018
})

In [6]:
dt.data

MemoryMappedTable
concept_set_idx: int32
concepts: list<item: string>
  child 0, item: string
target: string
----
concept_set_idx: [[0,0,0,0,1,...,269,270,270,270,270],[271,271,271,271,272,...,532,532,533,533,533],...,[752,753,753,753,753,...,988,988,988,988,988],[989,989,989,989,990,...,992,992,992,992,992]]
concepts: [[["field","look","stand"],["field","look","stand"],...,["sit","watch","ground"],["sit","watch","ground"]],[["book","sit","bed"],["book","sit","bed"],...,["score","field","goal","run"],["score","field","goal","run"]],...,[["tree","plant","ground","hole","dig"],["hit","player","run","base","ball"],...,["hold","pump","stand","car","nozzle"],["hold","pump","stand","car","nozzle"]],[["wheelbarrow","push","fall","sit","hill"],["wheelbarrow","push","fall","sit","hill"],...,["perform","wear","dance","stage","costume"],["perform","wear","dance","stage","costume"]]]
target: [["The player stood in the field looking at the batter.","The coach stands along the field, looking at the 

In [13]:
data = {}

In [14]:
for t in dt:
    id_ = t["concept_set_idx"]
    concepts = t["concepts"]
    target = t["target"]
    
    if id_ in data:
        data[id_]["targets"].append(target)
    else:
        data[id_] = {
            "concept_set_idx": id_,
            "concepts": concepts,
            "targets": [target],
        }

In [15]:
data[0]

{'concept_set_idx': 0,
 'concepts': ['field', 'look', 'stand'],
 'targets': ['The player stood in the field looking at the batter.',
  'The coach stands along the field, looking at the goalkeeper.',
  'I stood and looked across the field, peacefully.',
  'Someone stands, looking around the empty field.']}

In [17]:
d = Dataset.from_list(list(data.values()))

In [19]:
d[1]

{'concept_set_idx': 1,
 'concepts': ['kid', 'room', 'dance'],
 'targets': ['The silly kid loves to dance in her room.',
  'the dance kid  room is full of kids',
  'A kid is dancing in the room.',
  'A group of kids are dancing around a living room.']}

In [11]:
mult_ref = []
pred = []

In [16]:
for d in data.values():
    pred.append(d["targets"][0])
    mult_ref.append(d["targets"][1:])

In [17]:
from estimation_utils.evaluation import compute_common_gen_metrics

/home/vmeshchaninov/.conda/envs/fap2_env/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [25]:
from aac_metrics.functional import bleu

In [26]:
results = bleu(pred, mult_ref)

In [27]:
results

({'bleu_4': tensor(0.1803, dtype=torch.float64)},
 {'bleu_4': tensor([3.9281e-05, 1.0724e-12, 4.9394e-05, 6.8037e-01, 3.2948e-05, 8.0343e-05,
          6.4055e-13, 9.1932e-09, 8.3070e-09, 2.7829e-05, 4.5294e-09, 3.3032e-05,
          6.4662e-13, 7.8451e-13, 4.7193e-05, 3.1843e-05, 8.3070e-05, 7.0889e-13,
          3.9281e-05, 3.0935e-09, 7.7306e-09, 6.3878e-13, 9.6232e-13, 4.0041e-09,
          3.7685e-01, 4.5746e-09, 6.2390e-09, 4.5180e-05, 4.4693e-09, 5.8739e-05,
          3.9074e-09, 1.0724e-12, 4.9394e-09, 5.8739e-09, 5.3864e-09, 8.7836e-13,
          6.0580e-09, 8.0343e-05, 7.3861e-09, 1.0933e-08, 8.0343e-09, 4.0982e-09,
          7.5832e-09, 1.2132e-12, 3.7982e-09, 6.8037e-05, 3.3874e-01, 3.3141e-09,
          6.9853e-09, 3.6705e-01, 2.6658e-05, 9.6818e-09, 8.4090e-01, 3.3032e-01,
          3.9754e-09, 9.1287e-09, 9.4386e-13, 5.0933e-01, 4.5169e-09, 4.2836e-09,
          4.6714e-05, 5.9610e-09, 5.8093e-09, 3.5921e-09, 4.3748e-09, 1.4772e-12,
          6.0307e-09, 1.6467e-01, 5.14